<font size='6'><b>Advanced Arduino</b></font><br><br>

<table style="border-style: hidden; border-collapse: collapse;" width = "80%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 60% style="border-style: hidden; border-collapse: collapse;">

        </td>
        <td width = 20%>
        by Seungchul Lee<br>iSystems Design Lab<br>UNIST<br>http://isystems.unist.ac.kr/
        </td>
    </tr>
</table>

Table of Contents
<div id="toc"></div>

# 1. Visual Output (revisited)



## 1.1. Dual 7 Segment

Lab: 2 Digit 7 segment Counter from 0 to 99

dual 7 segments (개념이 아주 중요하다.)

https://github.com/mpthegoat/Arduino-digital-7-segment-counter-0-99

http://www.electroschematics.com/9636/arduino-segment-display-counter/

http://blog.rastating.com/using-a-7-segment-led-display-w-arduino-uno/


led lib (pointer)

http://arduvino.com/arduino-project-03-reading-temperature-using-the-lm35-sensor-and-displaying-it-on-a-7-segment-led-display-5261bs/

<table style="border-style: hidden; border-collapse: collapse;" width = "90%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 30% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/2_7seg.jpg" width = 250>
        </td>
        <td width = 30% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/two_7segs.jpg" width = 250>
        </td>
        <td width = 30%>
<img src = "./image_files/twodigit_7seg.jpg" width = 400>
        </td>
    </tr>
</table>

- In multiplexed LED applications the LED segments of all the digits are tied together and the common pins of each digit are turned ON separately by the microcontroller. 

- When each digit is displayed only for several milliseconds, the eye cannot tell that the digits are not ON all the time. This way we can multiplex any number of 7-segment displays together. 

- For example, to display the number 53, we have to send 5 to the first digit and enable its common pin. After a few milliseconds, number 3 is sent to the second digit and the common point of the second digit is enabled. When this process is repeated continuously, it appears to the user that both displays are ON continuously.

> 1) Send the segment bit pattern for digit 1 to segments $a$ to $g$

> 2) Enable digit 1

> 3) Wait for a few milliseconds

> 4) Disable digit 1 

> 5) Send the segment bit pattern for digit 2 to segments $a$ to $g$

> 6) Enable digit 2

> 7) Wait for a few milliseconds

> 8) Disable digit 2

> 9) Repeat


<img src = "./image_files/two_7seg_circuit.png" width = 350>

>need to add code

In [1]:
%%html
<iframe src="https://www.youtube.com/embed/1lA4UUCGUu0" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

In [2]:
%%html
<iframe src="https://www.youtube.com/embed/28PIhcRzs64" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

__Use LEDDisplay.h library__
```c
#include <LEDDisplay.h>

LEDDisplay *led;

void setup() {
  int digitFlagPins[] = {10, 11};
  int segmentPins[] = {2, 3, 4, 5 , 6 , 7 , 8, 9};
  int decimalPointPin = 9;
  led = new LEDDisplay(2, digitFlagPins, segmentPins, decimalPointPin);
}

void loop() {
  led -> displayNumber(2, 1);
  led -> displayNumber(4, 0);
}

```

To see this in a slightly more dynamic example, we can adapt the example to make the LED display act as a counter (obviously only up to the number 99). 

```c
#include <LEDDisplay.h>

LEDDisplay *led;
int millisecondsPerCount;
int counter;
unsigned long lastUpdate;

void setup()  {
  millisecondsPerCount = 1000;
  int digitFlagPins[] = {10, 11};
  int segmentPins[] = {2, 3, 4, 5 , 6 , 7 , 8, 9};
  int decimalPointPin = 9;
  led = new LEDDisplay(2, digitFlagPins, segmentPins, decimalPointPin);
}

void loop()  {
  unsigned long now = millis();
  if (now - lastUpdate > millisecondsPerCount) {
    lastUpdate = now;
    counter++;
    if (counter == 100) {
      counter = 0;
    }
  }

  int number = counter;
  for (int i = 0; i < 2; i++) {
    led -> displayNumber(number % 10, i);
    delay(2);
    number = number / 10;
  }
}
```

# 2. Interrupt (Revisited)

## 2.1. External Interrupt

https://www.sparkfun.com/tutorials/326


Interrupt handling

- Processor interrupts (preempts) the current flow of control

- Time spent in interrupt handlers should be kept as short as possible

- Microcontroller offers interrupts for various conditions
    - Not all are useful all the time: enable required interrupts
    - Some critical may require atomic execution (no interruptions guaranteed)
    - Disable / re-enable interrupts around critical section

Interrupts vs. Polling
- Polling:
    - Continuously poll IOs for change of value
    - Polling is like picking up your phone every few seconds to see if you have a call...
    - Cons:
        - Most polls are unneeded – value did not change
        - High CPU usage
        - Reaction time depends on #IOs

- Interrupt
    - Normal execution is interrupted when event occurs
    - Interrupts are like waiting for the phone to ring
    - Pro:
        - Processor resources are only used when necessary
    - Cons:
        - Program execution is interrupted in a non-deterministic manner


<img src = "./image_files/interrupt_flow_02.png" style="border:1px solid black", width =500>

Interrupt Service Routine (ISR)
- Event handler for interrupt
- Special, user-defined function for handling the interrupt


In [3]:
%%html
<iframe src="https://www.youtube.com/embed/yOMFFeCsmv8" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

```c
int pin = 13;           // LED on board
int interrupt0pin = 2;  // pin attached to int0 if you're using Uno
int buttonState;

void setup() {
  pinMode(pin, OUTPUT);
  pinMode(interrupt0pin, INPUT);

  // calls buttonUpdate() when button input changes (you can also use high-to-low or low-to-high)
  attachInterrupt(0, buttonUpdate, CHANGE);
}

void loop() {
  if (buttonState)
    digitalWrite(pin, HIGH);
  else
    digitalWrite(pin, LOW);
}

void buttonUpdate() {
  buttonState = digitalRead(interrupt0pin);
  delay(50);            // 50ms works for most switches - adjustable
}
```

<font size='4'><b>Lab: Interrupt-based debouncing</b></font>

```c
int pin = 13;           // LED on board
int interrupt0pin = 2;  // pin attached to int0 if you're using Uno
int buttonState;

void setup() {
  pinMode(pin, OUTPUT);
  pinMode(interrupt0pin, INPUT);

  // calls buttonUpdate() when button input changes (you can also use high-to-low or low-to-high)
  attachInterrupt(0, buttonUpdate, CHANGE);
}

void loop() {
  if (buttonState)
    digitalWrite(pin, HIGH);
  else
    digitalWrite(pin, LOW);
}

void buttonUpdate() {
  buttonState = digitalRead(interrupt0pin);
  delay(50);            // 50ms works for most switches - adjustable
}
```

motion detection sensor + interrupt

https://www.sparkfun.com/tutorials/326

http://www.bristolwatch.com/arduino/arduino_irq.htm
    

- You want to perform some action when a digital pin changes value and you don’t want to have to constantly check the pin state.

This sketch monitors pulses on pin 2 and stores the duration in an array. When the array has been filled (64 pulses have been received), the duration of each pulse is displayed on the Serial Monitor:

```c
/*
Interrupts sketch
see Recipe 10.1 for connection diagram from Arduino Cookbook
*/

const int irReceiverPin = 2; //pin the receiver is connected to
const int numberOfEntries = 64;
volatile unsigned long microseconds;
volatile byte index = 0;
volatile unsigned long results[numberOfEntries];

void setup() {
  pinMode(irReceiverPin, INPUT);
  Serial.begin(9600);
  attachInterrupt(0, analyze, CHANGE); // encoder pin on interrupt 0 (pin 2);
  results[0] = 0;
}
void loop() {
  if (index >= numberOfEntries) {
    Serial.println("Durations in Microseconds are:") ;
    for ( byte i = 0; i < numberOfEntries; i++) {
      Serial.println(results[i]);
    }
    while (1);
  }
  delay(1000);
}
void analyze() {
  if (index < numberOfEntries ) {
    if (index > 0) {
      results[index] = micros() - microseconds;
    }
    index = index + 1;
  }
  microseconds = micros();
}
```

## 2.2. timer interrupt

```c
#include "TimerOne.h"

void setup() {
  pinMode(13, OUTPUT);
  Timer1.initialize(500000);         // initialize timer1, and set a 500000 microseconds (or 1/2 second period)
  Timer1.attachInterrupt(timerIsr);  // attaches timerIsr() as a timer service routine
}

void timerIsr() {
  // Toggle LED
  digitalWrite(13, digitalRead(13) ^ 1);
}

void loop() {
  // your program here...
}
```

In [8]:
%%html
<iframe src="https://www.youtube.com/embed/m5_pFID-f-M" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

<font size='4'><b>Lab: LCD Timer</b></font>

```c
// include the library code:
#include <LiquidCrystal.h>

// initialize the library with the numbers of the interface pins
LiquidCrystal lcd(12, 11, 5, 4, 3, 2);

void setup() {
  // set up the LCD's number of columns and rows:
  lcd.begin(16, 2);
  // Print a message to the LCD.
  lcd.print("hello, world!");
}

void loop() {
  // set the cursor to column 0, line 1
  // (note: line 1 is the second row, since counting begins with 0):
  lcd.setCursor(0, 1);
  // print the number of seconds since reset:
  lcd.print(millis() / 1000);
}
```

<font size='4'><b>Lab: Dual 7 segments with time</b></font>

+ timer interrupt 에 대한 이해가 필요

https://www.youtube.com/watch?v=Y4qC8INSotM

# 3. SPI

https://learn.sparkfun.com/tutorials/serial-peripheral-interface-spi?_ga=1.137392624.1133730960.1469244666

# 4. I2C

In [2]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>